This notebook is used to determine which strategy is the best for any particular asset.

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd

df = pd.read_csv("../data/processed/starting_portfolio.csv")

In [4]:
import yfinance as yf

start_date = "2019-01-01"
end_date = "2024-12-31"

portfolio = {}
for asset, weight in list(map(list, df.values)):
    portfolio[asset] = {
        "data": yf.Ticker(asset).history(start=start_date, end=end_date, actions=False),
        "weight": weight,
        "strategy": None,
        "best_return": float("-inf"),
    }

## Testing with one asset and one strategy

In [ ]:
from backtesting import Backtest

from strategies.larry_williams_price_action import LarryWilliamsPriceAction

data = portfolio["BXP"]["data"]
data.index = data.index.values.astype("datetime64[D]")
bt = Backtest(
    data, LarryWilliamsPriceAction, cash=1000000, commission=0.002, exclusive_orders=True
)
stats = bt.optimize(
    size=[i / 100 for i in range(1, 10)],
    sl_pct=[i / 100 for i in range(1, 8)],
    tp_pct=[i / 100 for i in range(1, 8)],
    maximize="Return [%]",
    max_tries=3000,
    random_state=0,
)
# NOTE: Not sure how to get the optimized size/SL/TP values...
return_pct = stats["Return [%]"]
if return_pct >= portfolio["BXP"]["best_return"]:
    portfolio["BXP"]["strategy"] = bt._strategy.__name__
    portfolio["BXP"]["best_return"] = return_pct

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_stats.py:157: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_matrix = np.cov(equity_log_returns, market_log_returns)
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12

In [7]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag
0,-902,26,35,99.737843,102.151627,106.363768,98.336314,-2177.233058,-0.024201,2019-02-08,2019-02-22,14 days,None
1,-878,35,38,102.151627,99.782336,107.927833,99.782336,2080.236953,0.023194,2019-02-22,2019-02-27,5 days,None
2,869,81,82,103.439155,105.243608,96.989207,105.243608,1568.069725,0.017445,2019-04-30,2019-05-01,1 days,None
3,900,136,138,100.017212,98.128368,93.929560,101.923565,-1699.959663,-0.018885,2019-07-18,2019-07-22,4 days,None
4,915,138,141,98.128368,100.302399,92.435545,100.302399,1989.238408,0.022155,2019-07-22,2019-07-25,3 days,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1414,1399,1399,65.439269,66.638680,61.412117,66.638680,1695.966000,0.018329,2024-07-25,2024-07-25,0 days,None
85,-1107,1456,1459,83.709149,87.833920,88.271461,81.609464,-4566.122132,-0.049275,2024-10-15,2024-10-18,3 days,None
86,-1050,1459,1460,87.833920,85.990200,93.009808,85.990200,1935.906476,0.020991,2024-10-18,2024-10-21,3 days,None
87,1151,1476,1487,80.255392,82.761832,76.024444,82.494609,2884.912286,0.031231,2024-11-12,2024-11-27,15 days,None


## All assets and all strategies

In [ ]:
from backtesting import Backtest

from strategies.larry_williams_price_action import LarryWilliamsPriceAction
from strategies.macd_bollinger_bands_mean_reversion import MACDBollingerBandsMeanReversion
from strategies.bollinger_bands_breakout import BollingerBandsBreakout

# TODO: Add more
strategies = [LarryWilliamsPriceAction, MACDBollingerBandsMeanReversion, BollingerBandsBreakout]

for asset in portfolio:
    for strategy in strategies:
        data = portfolio[asset]["data"]
        data.index = data.index.values.astype("datetime64[D]")
        bt = Backtest(
            data, strategy, cash=1000000, commission=0.002, exclusive_orders=True
        )
        stats = bt.optimize(
            size=[i / 100 for i in range(1, 10)],
            sl_pct=[i / 100 for i in range(1, 8)],
            tp_pct=[i / 100 for i in range(1, 8)],
            maximize="Return [%]",
            max_tries=3000,
            random_state=0,
        )
        # NOTE: Not sure how to get the optimized size/SL/TP values...
        return_pct = stats["Return [%]"]
        if return_pct >= portfolio[asset]["best_return"]:
            portfolio[asset]["strategy"] = bt._strategy.__name__
            portfolio[asset]["best_return"] = return_pct

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_stats.py:157: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_matrix = np.cov(equity_log_returns, market_log_returns)
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
Backtest.optimize:   0%|          | 0/49 [00:00<?, ?it/s]

In [11]:
results = pd.DataFrame(
    [
        [
            asset,
            portfolio[asset]["weight"],
            portfolio[asset]["strategy"],
            portfolio[asset]["best_return"],
        ]
        for asset in portfolio
    ],
    columns=["asset", "weight", "strategy", "best_return"]
)

results.to_csv("../data/misc/example_backtesting.csv", index=False)